In [3]:
import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem
from molvecgen.vectorizers import SmilesVectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [4]:
# Load and preprocess data
df = pd.read_csv('small_molecules_smiles_only_for_deep_learning_sm_10032024.csv')

In [5]:
PandasTools.AddMoleculeColumnToFrame(df, 'canon_smiles', 'Molecule')
df.head()

[09:57:44] WARNING: not removing hydrogen atom without neighbors
[09:57:44] WARNING: not removing hydrogen atom without neighbors


,good,canon_smiles,Molecule
0,0,CC1(C)CCC(c2ccc(Cl)cc2)=C(CN2CCN(c3ccc(C(=O)NS...,<rdkit.Chem.rdchem.Mol object at 0x0000020180D...
1,0,CN(C)CC[C@H](CSc1ccccc1)Nc1ccc(S(=O)(=O)NC(=O)...,<rdkit.Chem.rdchem.Mol object at 0x0000020180D...
2,0,Cc1ccc(F)c(NC(=O)Nc2ccc(-c3cccc4[nH]nc(N)c34)c...,<rdkit.Chem.rdchem.Mol object at 0x0000020180D...
3,0,C[C@@]1(c2nc3c(C(N)=O)cccc3[nH]2)CCCN1,<rdkit.Chem.rdchem.Mol object at 0x0000020180D...
4,0,CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1,<rdkit.Chem.rdchem.Mol object at 0x0000020180D...


In [6]:
smivec = SmilesVectorizer(pad=1, leftpad=True, canonical=False, augment=True)
smivec.fit(df.Molecule.values)

In [7]:
print(smivec.charset)
smivec.dims

Z]oNu7.#e+)42\(5t1cS-C@3HRsBMKPan/rF[I6l=O^$?


(275, 45)

In [8]:
y = df['good'].values.reshape((-1, 1)).astype(np.float32)  # Ensure y is float32
X = df.Molecule.values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state=42)

In [10]:
# The class wraps the SMILES data and their corresponding labels into a format compatible with PyTorch 'DataLoader'.
# It uses the SMILES vectorizer to convert the SMILES strings into numeric vectors during data retrieval

class SMILESMolDataset(Dataset):
    def __init__(self, molecules, y, vectorizer):
        self.molecules = molecules
        self.y = y
        self.vectorizer = vectorizer

    def __len__(self):
        return len(self.molecules)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        mols = self.molecules[idx]

        # Transform the SMILES string into a vector
        sample = self.vectorizer.transform([mols])[0]
        label = self.y[idx]

        # Ensure sample is converted to float32
        sample = torch.tensor(sample, dtype=torch.float32)

        return sample, torch.tensor(label, dtype=torch.float32)  # Also convert label to float32

In [11]:
train_dataset = SMILESMolDataset(X_train, y_train, smivec)

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [11]:
X_val_t = smivec.transform(X_val, canonical=False)
X_val_t = torch.tensor(X_val_t, device=device, dtype=torch.float32)  # Set dtype to float32
y_val_t = torch.tensor(y_val, device=device, dtype=torch.float32)  # Set dtype to float32

In [12]:
class Net(nn.Module):
    def __init__(self, dimensions, lstm_sizes, hidden_size, dropout_rate, out_size=1):
        super(Net, self).__init__()

        number_tokens = dimensions[1]

        # LSTM layers list
        self.lstms = nn.ModuleList()

        input_size = number_tokens
        for lstm_size in lstm_sizes:
            self.lstms.append(nn.LSTM(input_size=input_size, hidden_size=lstm_size, num_layers=1, batch_first=True))
            input_size = lstm_size

        # Only add dropout if there's more than 1 LSTM layer
        if len(lstm_sizes) > 1:
            self.dropouts = nn.Dropout(dropout_rate)
        else:
            self.dropouts = None  # No dropout needed for a single LSTM layer

        # Define fully connected layers
        self.fc1 = nn.Linear(lstm_sizes[-1], hidden_size)  # Hidden layer
        self.activation = nn.ReLU()  # Activation function for hidden layer
        self.fc_out = nn.Linear(hidden_size, out_size)  # Output layer

    def forward(self, x):
        # Forward pass through LSTM layers
        for lstm in self.lstms:
            x, _ = lstm(x)

        # Apply dropout if there is more than 1 LSTM layer
        if self.dropouts:
            x = self.dropouts(x)

        # Use the last hidden state from the last LSTM layer
        x = x[:, -1, :]

        # Pass through the hidden layer and activation function
        x = self.fc1(x)
        x = self.activation(x)

        # Final output layer
        x = self.fc_out(x)

        return x

In [13]:
epochs = 50  # I can implement early stopping to halt training when the validation loss stops improving.
dims = smivec.dims  # This is necessary to define the input dimensions. Ensure 'smivec.dims' returns the correct shape: '[sequence_length, number_tokens]'.
lstm_size = [32, 16]  # The size of the LSTM hidden state. I may experiment with larger size (e.g., 256 or 512) if necessary.
hidden_size = 64  # This number is a typical one. I'll adjust it based on the model's performance
dropout_rate = 0.5  # A dropout rate of 0.5 is standard for preventing overfitting. I may reduce it to 0.3 if underfitting occurs.
output_size = 1  # This is for binary classification, as the output is a single probability value.
batch_size = 12  # A batch size of 128 is commonly used and generally effective. Depending on GPU memory, I may try different sizes (e.g., 64 or 256).
learning_rate = 0.0005  # If the training is unstable, I'll lower it to 0.001.

In [14]:
model = Net(dims, lstm_size, hidden_size, dropout_rate, output_size).to(device)
model

Net(
  (lstms): ModuleList(
    (0): LSTM(45, 32, batch_first=True)
    (1): LSTM(32, 16, batch_first=True)
  )
  (dropouts): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=16, out_features=64, bias=True)
  (activation): ReLU()
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
)

In [15]:
y_train.shape

(1080, 1)

In [16]:
from sklearn.utils.class_weight import compute_class_weight

# Ensure y_train is a 1D array
y_train_flat = np.ravel(y_train)  # Flatten y_train if needed

# Compute class weights based on your labels
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_flat), y=y_train_flat)
class_weights = torch.FloatTensor(class_weights).to(device)

In [17]:
class_weights

tensor([ 0.5118, 21.6000], device='cuda:0')

In [18]:
class_weights[1]

tensor(21.6000, device='cuda:0')

In [19]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [20]:
# results = {}

# for pos_weight_value in custom_pos_weights:
#     class_weights = torch.FloatTensor([1.0, pos_weight_value]).to(device)  # Set pos_weight for binary class

#     # Define criterion with the chosen pos_weight
#     criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#     lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

#     train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#     train_losses, validation_losses = [], []

#     # Early stopping setup
#     early_stopping_patience = 10
#     best_val_loss = float('inf')
#     epochs_without_improvement = 0

#     start_time = time.time()

#     for epoch in range(epochs):
#         running_loss = 0.0
#         model.train()
        
#         for smiles, labels in train_loader:
#             smiles = smiles.clone().detach().float().to(device)
#             labels = labels.clone().detach().float().to(device)
            
#             optimizer.zero_grad()
#             output = model(smiles)
#             loss = criterion(output, labels)
#             loss.backward()
#             torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
#             optimizer.step()
            
#             running_loss += loss.item()
        
#         # Validation step
#         model.eval()
#         with torch.no_grad():
#             val_outputs = model(X_val_t)
#             val_loss = criterion(val_outputs, y_val_t).item()
        
#         train_loss = running_loss / len(train_loader)
#         train_losses.append(train_loss)
#         validation_losses.append(val_loss)

#         lr_scheduler.step(val_loss)

#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             epochs_without_improvement = 0
#             torch.save(model.state_dict(), f'best_model_pos_weight_{pos_weight_value}.pth')
#         else:
#             epochs_without_improvement += 1

#         if epochs_without_improvement >= early_stopping_patience:
#             print(f'Early stopping at epoch {epoch + 1}')
#             break

#         if (epoch + 1) % 2 == 0:
#             print(f'Pos_weight: {pos_weight_value}, Epoch {epoch+1}, Training loss: {train_loss:.2f}, Validation loss: {val_loss:.2f}')
        
#         torch.cuda.empty_cache()

#     total_time = time.time() - start_time
#     results[pos_weight_value] = {
#         'train_losses': train_losses,
#         'validation_losses': validation_losses,
#         'best_val_loss': best_val_loss,
#         'training_time': total_time / 60
#     }
#     print(f'Pos_weight: {pos_weight_value}, Total training time: {total_time/60:.2f} mins')

# # After training, analyze results for each pos_weight
# for pos_weight, result in results.items():
#     print(f"\nPos_weight {pos_weight}: Best Validation Loss = {result['best_val_loss']:.4f}")

# Try to use optuna and ray for hyperparameters tuning

In [35]:
# Using PyTorch
import torch
num_gpus = torch.cuda.device_count()
num_gpus

1

In [37]:
import multiprocessing
num_cpus = multiprocessing.cpu_count()
num_cpus

24

In [17]:
import rdkit
from rdkit import Chem
from rdkit.Chem import PandasTools, AllChem
from molvecgen.vectorizers import SmilesVectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
import ray
from ray import tune
from ray import train
from ray import air
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
from ray.air import RunConfig
from ray.tune.logger import JsonLogger, CSVLogger

import os

# Create the directory if it doesn't exist
storage_path = os.path.abspath("C:/RayResults")
os.makedirs(storage_path, exist_ok=True)

# Disable TensorBoard logging
os.environ["TUNE_DISABLE_AUTO_CALLBACK_LOGGERS"] = "1"

# Load and preprocess data
df = pd.read_csv('small_molecules_smiles_only_for_deep_learning_sm_10032024.csv')
PandasTools.AddMoleculeColumnToFrame(df, 'canon_smiles', 'Molecule')

smivec = SmilesVectorizer(pad=1, leftpad=True, canonical=False, augment=True)
smivec.fit(df.Molecule.values)

y = df['good'].values.reshape((-1, 1)).astype(np.float32)
X = df.Molecule.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

class SMILESMolDataset(Dataset):
    def __init__(self, molecules, y, vectorizer):
        self.molecules = molecules
        self.y = y
        self.vectorizer = vectorizer

    def __len__(self):
        return len(self.molecules)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        mols = self.molecules[idx]
        sample = self.vectorizer.transform([mols])[0]
        label = self.y[idx]

        sample = torch.tensor(sample, dtype=torch.float32)
        return sample, torch.tensor(label, dtype=torch.float32)

train_dataset = SMILESMolDataset(X_train, y_train, smivec)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

X_val_t = smivec.transform(X_val, canonical=False)
X_val_t = torch.tensor(X_val_t, device=device, dtype=torch.float32)
y_val_t = torch.tensor(y_val, device=device, dtype=torch.float32)

class Net(nn.Module):
    def __init__(self, dimensions, lstm_sizes, hidden_size, dropout_rate, out_size=1):
        super(Net, self).__init__()

        number_tokens = dimensions[1]

        self.lstms = nn.ModuleList()

        input_size = number_tokens
        for lstm_size in lstm_sizes:
            self.lstms.append(nn.LSTM(input_size=input_size, hidden_size=lstm_size, num_layers=1, batch_first=True))
            input_size = lstm_size

        self.dropouts = nn.Dropout(dropout_rate) if len(lstm_sizes) > 1 else None

        self.fc1 = nn.Linear(lstm_sizes[-1], hidden_size)
        self.activation = nn.ReLU()
        self.fc_out = nn.Linear(hidden_size, out_size)

    def forward(self, x):
        for lstm in self.lstms:
            x, _ = lstm(x)

        if self.dropouts:
            x = self.dropouts(x)

        x = x[:, -1, :]
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc_out(x)

        return x

# Global variables
dims = smivec.dims
lstm_size = [32, 16]
hidden_size = 64
dropout_rate = 0.5
output_size = 1
batch_size = 12

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

def train_model(config):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Net(dims, 
                config['lstm_size'], 
                config['hidden_size'], 
                config['dropout_rate'], 
                output_size).to(device)
    
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([config["pos_weight"]], device=device))
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                               batch_size=config['batch_size'], 
                                               shuffle=True)

    best_val_loss = float("inf")
    for epoch in range(config["epochs"]):
        model.train()
        running_loss = 0.0
        for smiles, labels in train_loader:
            smiles, labels = smiles.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(smiles)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_t.to(device))
            val_loss = criterion(val_outputs, y_val_t.to(device)).item()
            if val_loss < best_val_loss:
                best_val_loss = val_loss

        # Report metrics
        train.report({"val_loss": val_loss, "training_loss": running_loss / len(train_loader)})
    
    # Save final results
    train.save_checkpoint({"best_val_loss": best_val_loss})

def short_dirname_creator(trial):
    return f"trial_{trial.trial_id[:8]}"

print(device)

def tune_hyperparameters():
    if not ray.is_initialized():
        num_gpus = torch.cuda.device_count()
        ray.init(num_cpus=24, num_gpus=num_gpus, ignore_reinit_error=True)
        
    storage_path = os.path.abspath("C:/RT")  # Shorter path
    os.makedirs(storage_path, exist_ok=True)

    config = {
        "hidden_size": tune.choice([32, 64, 128, 256]),
        "lstm_size": tune.choice([[32, 16], [64, 32], [64, 32, 16], [128, 64, 32], [256, 128, 64], [256, 128, 32], [256, 128, 64, 32]]),
        "dropout_rate": tune.loguniform(0.2, 0.8),
        "learning_rate": tune.loguniform(1e-4, 1e-2),
        "batch_size": tune.choice([4, 8, 16, 32, 64]),
        "epochs": tune.randint(10, 50),
        "pos_weight": tune.randint(1, 100)
    }

    scheduler = ASHAScheduler(
        max_t=50,
        grace_period=1,
        reduction_factor=2)

    search_alg = OptunaSearch()

    storage_path = os.path.abspath("C:/RayResults")
    os.makedirs(storage_path, exist_ok=True)

    tuner = tune.Tuner(
        train_model,
        tune_config=tune.TuneConfig(
            metric="val_loss",
            mode="min",
            scheduler=scheduler,
            search_alg=search_alg,
            num_samples=50,
            trial_dirname_creator=short_dirname_creator,
        ),
        param_space=config,
        run_config=air.RunConfig(
            name="tune_experiment",
            storage_path=storage_path,
        )
    )

    results = tuner.fit()
    
    best_result = results.get_best_result("val_loss", "min")
    print("Best trial config:", best_result.config)
    print("Best trial final validation loss:", best_result.metrics["val_loss"])

    ray.shutdown()

if __name__ == "__main__":
    tune_hyperparameters()

(raylet) Warning: The actor ImplicitFunc is very large (13 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


2024-10-26 10:13:04,366	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_254d63a4
Traceback (most recent call last):
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\worker.py", line 2745, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\worker.py", line 903, in get_objects
    raise value
ray.exceptions.ActorDiedError: The actor died unexpectedly before finishing this task.
	class_name: wrap_function.<locals>.Im

(raylet) Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 1856, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1957, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1862, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1803, in ray._raylet.execute_task.function_executor
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 696, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 608, in temporary_actor_method
    raise RuntimeError(
RuntimeError: The actor with name ImplicitFunc failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

Traceback (most recent call last):
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 647, in _load_actor_class_from_gcs
 

2024-10-26 10:13:07,780	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_3041666e
Traceback (most recent call last):
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\worker.py", line 2745, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\worker.py", line 903, in get_objects
    raise value
ray.exceptions.ActorDiedError: The actor died unexpectedly before finishing this task.
	class_name: wrap_function.<locals>.Im

(raylet) Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 1856, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1957, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1862, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1803, in ray._raylet.execute_task.function_executor
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 696, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 608, in temporary_actor_method
    raise RuntimeError(
RuntimeError: The actor with name ImplicitFunc failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

Traceback (most recent call last):
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 647, in _load_actor_class_from_gcs
 

...

(raylet) Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 1856, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1957, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1862, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1803, in ray._raylet.execute_task.function_executor
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 696, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 608, in temporary_actor_method
    raise RuntimeError(
RuntimeError: The actor with name ImplicitFunc failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

Traceback (most recent call last):
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 647, in _load_actor_class_from_gcs
 

2024-10-26 10:14:13,561	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_2d68fb9d
Traceback (most recent call last):
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\worker.py", line 2745, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\worker.py", line 903, in get_objects
    raise value
ray.exceptions.ActorDiedError: The actor died unexpectedly before finishing this task.
	class_name: wrap_function.<locals>.Im

(raylet) Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 1856, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1957, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1862, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1803, in ray._raylet.execute_task.function_executor
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 696, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 608, in temporary_actor_method
    raise RuntimeError(
RuntimeError: The actor with name ImplicitFunc failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

Traceback (most recent call last):
  File "C:\Users\mashu\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 647, in _load_actor_class_from_gcs
 

2024-10-26 10:14:18,722	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-10-26 10:14:18,742	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/RayResults/tune_experiment' in 0.0205s.
2024-10-26 10:14:19,151	ERROR tune.py:1037 -- Trials did not complete: [train_model_254d63a4, train_model_3041666e, train_model_49bf2b64, train_model_d5c545d8, train_model_6bf6bedc, train_model_88ec23b2, train_model_317aff42, train_model_98ec385a, train_model_3dcf099c, train_model_5b058ad5, train_model_75e89fa9, train_model_9134b7d5, train_model_ef02a77d, train_model_010ada71, train_model_4758663c, train_model_f571f4c0, train_model_9a1f2ab2, train_model_12f8d419, train_model_1d04ba99, train_model_46221966, train_model_d34d1d3c, train_model_909de791, train_model_0a9044e3, train_model_2d68fb9d, train_m

RuntimeError: No best trial found for the given metric: val_loss. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.

# Use regular gridsearchCV to tune hyperparameters (it can work but take too long time)

### lstm_sizes, hidden_size, dropout_rate, learning_rate, batch_size, pos_weight

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import PandasTools, AllChem
from molvecgen.vectorizers import SmilesVectorizer
from sklearn.base import BaseEstimator

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
import time  # Import time for timing the training process

import numpy as np
import pandas as pd
import os

# Load and preprocess data
df = pd.read_csv('small_molecules_smiles_only_for_deep_learning_sm_10032024.csv')
PandasTools.AddMoleculeColumnToFrame(df, 'canon_smiles', 'Molecule')

smivec = SmilesVectorizer(pad=1, leftpad=True, canonical=False, augment=True)
smivec.fit(df.Molecule.values)

y = df['good'].values.reshape((-1, 1)).astype(np.float32)
X = df.Molecule.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

class SMILESMolDataset(Dataset):
    def __init__(self, molecules, y, vectorizer):
        self.molecules = molecules
        self.y = y
        self.vectorizer = vectorizer

    def __len__(self):
        return len(self.molecules)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        mols = self.molecules[idx]
        sample = self.vectorizer.transform([mols])[0]
        label = self.y[idx]

        sample = torch.tensor(sample, dtype=torch.float32)
        return sample, torch.tensor(label, dtype=torch.float32)

class Net(nn.Module):
    def __init__(self, dimensions, lstm_sizes, hidden_size, dropout_rate, out_size=1):
        super(Net, self).__init__()

        number_tokens = dimensions[1]

        self.lstms = nn.ModuleList()
        input_size = number_tokens
        for lstm_size in lstm_sizes:
            self.lstms.append(nn.LSTM(input_size=input_size, hidden_size=lstm_size, num_layers=1, batch_first=True))
            input_size = lstm_size

        self.dropouts = nn.Dropout(dropout_rate) if len(lstm_sizes) > 1 else None
        self.fc1 = nn.Linear(lstm_sizes[-1], hidden_size)
        self.activation = nn.ReLU()
        self.fc_out = nn.Linear(hidden_size, out_size)

    def forward(self, x):
        for lstm in self.lstms:
            x, _ = lstm(x)

        if self.dropouts:
            x = self.dropouts(x)

        x = x[:, -1, :]
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc_out(x)

        return x

# Define a custom estimator class for use with GridSearchCV

class PyTorchClassifier(BaseEstimator):
    def __init__(self, vectorizer, lstm_sizes, hidden_size, dropout_rate, learning_rate, batch_size, pos_weight, patience=10, min_delta=0.01):
        self.vectorizer = vectorizer
        self.lstm_sizes = lstm_sizes
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.pos_weight = pos_weight
        self.model = None
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.patience = patience
        self.min_delta = min_delta

    def fit(self, X, y):
        # Prepare dataset
        dataset = SMILESMolDataset(X, y, self.vectorizer)
        train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=self.batch_size, shuffle=True)

        # Initialize model
        self.model = Net(dims, self.lstm_sizes, self.hidden_size, self.dropout_rate).to(self.device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([self.pos_weight], device=self.device))
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

        # Start timing
        start_time = time.time()

        best_val_loss = float('inf')
        epochs_without_improvement = 0

        # Training loop
        for epoch in range(50):  # Total epochs set to 50
            self.model.train()
            running_loss = 0.0
            for smiles, labels in train_loader:
                smiles, labels = smiles.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                output = self.model(smiles)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            # Calculate training loss
            train_loss = running_loss / len(train_loader)

            # Validation
            self.model.eval()
            with torch.no_grad():
                val_loader = torch.utils.data.DataLoader(dataset=SMILESMolDataset(X_val, y_val, self.vectorizer), 
                                                          batch_size=self.batch_size, 
                                                          shuffle=False)
                val_running_loss = 0.0
                for smiles, labels in val_loader:
                    smiles, labels = smiles.to(self.device), labels.to(self.device)
                    val_outputs = self.model(smiles)
                    val_loss = criterion(val_outputs, labels)
                    val_running_loss += val_loss.item()
                valid_loss = val_running_loss / len(val_loader)

            # Print losses and parameters every 5 epochs
            if (epoch + 1) % 4 == 0:
                print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}')
                print(f'Used Parameters: lstm_sizes={self.lstm_sizes}, hidden_size={self.hidden_size}, '
                      f'dropout_rate={self.dropout_rate}, learning_rate={self.learning_rate}, '
                      f'batch_size={self.batch_size}, pos_weight={self.pos_weight}')

            # Early stopping logic
            if valid_loss < best_val_loss - self.min_delta:
                best_val_loss = valid_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= self.patience:
                    print("Early stopping activated!")
                    break

        # Calculate total time
        total_time = time.time() - start_time
        print(f'Total training time: {total_time:.2f} seconds')

    def predict(self, X):
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(self.vectorizer.transform(X), dtype=torch.float32).to(self.device)
            outputs = self.model(X_tensor)
            preds = torch.sigmoid(outputs).cpu().numpy()
            return (preds > 0.5).astype(int)

    def get_params(self, deep=True):
        return {
            'vectorizer': self.vectorizer,
            'lstm_sizes': self.lstm_sizes,
            'hidden_size': self.hidden_size,
            'dropout_rate': self.dropout_rate,
            'learning_rate': self.learning_rate,
            'batch_size': self.batch_size,
            'pos_weight': self.pos_weight,
            'patience': self.patience,
            'min_delta': self.min_delta
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self
    
# Hyperparameter grid
param_grid = {
    'lstm_sizes': [[32, 16], [64, 32], [64, 32, 16]],
    'hidden_size': [32, 64, 128],
    'dropout_rate': [0.3, 0.5, 0.7],
    'learning_rate': [0.01, 0.001, 0.0005, 0.0001],
    'batch_size': [4, 8, 16],
    'pos_weight': [1, 10, 100]
}

# Create a scorer for evaluation
def custom_scorer(y_true, y_pred):
    return -np.mean(np.abs(y_true - y_pred))  # Custom scoring can be adjusted

scorer = make_scorer(custom_scorer, greater_is_better=False)

# Prepare the GridSearchCV
# Assuming `smivec` is your vectorizer instance
grid_search = GridSearchCV(
    estimator=PyTorchClassifier(
        vectorizer=smivec,      # Pass the vectorizer here
        lstm_sizes=[32, 16],    # Placeholder; will be overridden by grid search
        hidden_size=64,         # Placeholder; will be overridden by grid search
        dropout_rate=0.5,       # Placeholder; will be overridden by grid search
        learning_rate=1e-4,     # Placeholder; will be overridden by grid search
        batch_size=4,           # Placeholder; will be overridden by grid search
        pos_weight=1            # Placeholder; will be overridden by grid search
    ),
    param_grid=param_grid,
    scoring=scorer,
    cv=3  # Number of cross-validation folds
)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Print best parameters and score
print("Best parameters found: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Predict and evaluate on test set
y_pred = grid_search.predict(X_test)

[10:45:13] WARNING: not removing hydrogen atom without neighbors
[10:45:13] WARNING: not removing hydrogen atom without neighbors


Epoch 4, Train Loss: 0.1078, Valid Loss: 0.1078
Used Parameters: lstm_sizes=[32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.01, batch_size=4, pos_weight=1
Epoch 8, Train Loss: 0.1074, Valid Loss: 0.0925
Used Parameters: lstm_sizes=[32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.01, batch_size=4, pos_weight=1
Epoch 12, Train Loss: 0.1104, Valid Loss: 0.0930
Used Parameters: lstm_sizes=[32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.01, batch_size=4, pos_weight=1
Early stopping activated!
Total training time: 7.43 seconds
Epoch 4, Train Loss: 0.1277, Valid Loss: 0.0928
Used Parameters: lstm_sizes=[32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.01, batch_size=4, pos_weight=1
Epoch 8, Train Loss: 0.1252, Valid Loss: 0.0958
Used Parameters: lstm_sizes=[32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.01, batch_size=4, pos_weight=1
Early stopping activated!
Total training time: 6.74 seconds
Epoch 4, Train Loss: 0.1239, Valid Loss: 0.0

Early stopping activated!
Total training time: 8.90 seconds
Epoch 4, Train Loss: 2.0334, Valid Loss: 1.8980
Used Parameters: lstm_sizes=[64, 32], hidden_size=32, dropout_rate=0.3, learning_rate=0.01, batch_size=4, pos_weight=100
Epoch 8, Train Loss: 2.1459, Valid Loss: 1.8249
Used Parameters: lstm_sizes=[64, 32], hidden_size=32, dropout_rate=0.3, learning_rate=0.01, batch_size=4, pos_weight=100
Epoch 12, Train Loss: 2.1011, Valid Loss: 1.8466
Used Parameters: lstm_sizes=[64, 32], hidden_size=32, dropout_rate=0.3, learning_rate=0.01, batch_size=4, pos_weight=100
Early stopping activated!
Total training time: 8.37 seconds
Epoch 4, Train Loss: 2.1170, Valid Loss: 1.8259
Used Parameters: lstm_sizes=[64, 32], hidden_size=32, dropout_rate=0.3, learning_rate=0.01, batch_size=4, pos_weight=100
Epoch 8, Train Loss: 2.0873, Valid Loss: 1.8309
Used Parameters: lstm_sizes=[64, 32], hidden_size=32, dropout_rate=0.3, learning_rate=0.01, batch_size=4, pos_weight=100
Epoch 12, Train Loss: 2.0785, Vali

Epoch 4, Train Loss: 2.0337, Valid Loss: 1.8579
Used Parameters: lstm_sizes=[32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.001, batch_size=4, pos_weight=100
Epoch 8, Train Loss: 1.8488, Valid Loss: 1.8497
Used Parameters: lstm_sizes=[32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.001, batch_size=4, pos_weight=100
Epoch 12, Train Loss: 1.9358, Valid Loss: 1.8243
Used Parameters: lstm_sizes=[32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.001, batch_size=4, pos_weight=100
Epoch 16, Train Loss: 1.9516, Valid Loss: 1.8122
Used Parameters: lstm_sizes=[32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.001, batch_size=4, pos_weight=100
Early stopping activated!
Total training time: 9.84 seconds
Epoch 4, Train Loss: 2.1798, Valid Loss: 1.8210
Used Parameters: lstm_sizes=[32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.001, batch_size=4, pos_weight=100
Epoch 8, Train Loss: 2.1260, Valid Loss: 1.8323
Used Parameters: lstm_sizes=[32, 16], hi

Epoch 8, Train Loss: 0.5979, Valid Loss: 0.5139
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.001, batch_size=4, pos_weight=10
Early stopping activated!
Total training time: 8.83 seconds
Epoch 4, Train Loss: 1.9125, Valid Loss: 1.8413
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.001, batch_size=4, pos_weight=100
Epoch 8, Train Loss: 1.9199, Valid Loss: 1.8228
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.001, batch_size=4, pos_weight=100
Epoch 12, Train Loss: 1.9290, Valid Loss: 1.8256
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.001, batch_size=4, pos_weight=100
Early stopping activated!
Total training time: 9.69 seconds
Epoch 4, Train Loss: 2.1303, Valid Loss: 1.8456
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=32, dropout_rate=0.3, learning_rate=0.001, batch_size=4, pos_weight=100
Epoch 8, T

...

Early stopping activated!
Total training time: 7.76 seconds
Epoch 4, Train Loss: 2.0136, Valid Loss: 1.8274
Used Parameters: lstm_sizes=[64, 32], hidden_size=64, dropout_rate=0.3, learning_rate=0.0005, batch_size=8, pos_weight=100
Epoch 8, Train Loss: 1.9749, Valid Loss: 1.9760
Used Parameters: lstm_sizes=[64, 32], hidden_size=64, dropout_rate=0.3, learning_rate=0.0005, batch_size=8, pos_weight=100
Epoch 12, Train Loss: 2.1090, Valid Loss: 1.8296
Used Parameters: lstm_sizes=[64, 32], hidden_size=64, dropout_rate=0.3, learning_rate=0.0005, batch_size=8, pos_weight=100
Epoch 16, Train Loss: 2.0436, Valid Loss: 1.8376
Used Parameters: lstm_sizes=[64, 32], hidden_size=64, dropout_rate=0.3, learning_rate=0.0005, batch_size=8, pos_weight=100
Early stopping activated!
Total training time: 6.95 seconds
Epoch 4, Train Loss: 0.1060, Valid Loss: 0.0917
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0005, batch_size=8, pos_weight=1
Epoch 8, Train Loss: 

Epoch 4, Train Loss: 0.1728, Valid Loss: 0.1268
Used Parameters: lstm_sizes=[32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Epoch 8, Train Loss: 0.1218, Valid Loss: 0.0940
Used Parameters: lstm_sizes=[32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Epoch 12, Train Loss: 0.1190, Valid Loss: 0.0925
Used Parameters: lstm_sizes=[32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Epoch 16, Train Loss: 0.1193, Valid Loss: 0.0922
Used Parameters: lstm_sizes=[32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Early stopping activated!
Total training time: 6.61 seconds
Epoch 4, Train Loss: 0.6206, Valid Loss: 0.5545
Used Parameters: lstm_sizes=[32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=10
Epoch 8, Train Loss: 0.5604, Valid Loss: 0.5098
Used Parameters: lstm_sizes=[32, 16], hidden

Epoch 8, Train Loss: 0.1030, Valid Loss: 0.0918
Used Parameters: lstm_sizes=[64, 32], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Epoch 12, Train Loss: 0.1018, Valid Loss: 0.0917
Used Parameters: lstm_sizes=[64, 32], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Early stopping activated!
Total training time: 5.24 seconds
Epoch 4, Train Loss: 0.1275, Valid Loss: 0.0965
Used Parameters: lstm_sizes=[64, 32], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Epoch 8, Train Loss: 0.1242, Valid Loss: 0.0925
Used Parameters: lstm_sizes=[64, 32], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Epoch 12, Train Loss: 0.1267, Valid Loss: 0.0923
Used Parameters: lstm_sizes=[64, 32], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Early stopping activated!
Total training time: 5.56 seconds
Epoch 4, Train Loss: 0.1270, Vali

Epoch 16, Train Loss: 0.1094, Valid Loss: 0.0918
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Early stopping activated!
Total training time: 7.24 seconds
Epoch 4, Train Loss: 0.1901, Valid Loss: 0.1375
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Epoch 8, Train Loss: 0.1282, Valid Loss: 0.0952
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Epoch 12, Train Loss: 0.1205, Valid Loss: 0.0927
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Epoch 16, Train Loss: 0.1257, Valid Loss: 0.0925
Used Parameters: lstm_sizes=[64, 32, 16], hidden_size=64, dropout_rate=0.3, learning_rate=0.0001, batch_size=8, pos_weight=1
Early stopping activated!
Total training time: 6.90 seconds
Epoch 4, Tra

Epoch 16, Train Loss: 0.1219, Valid Loss: 0.0930
Used Parameters: lstm_sizes=[32, 16], hidden_size=128, dropout_rate=0.3, learning_rate=0.01, batch_size=8, pos_weight=1
Epoch 20, Train Loss: 0.1197, Valid Loss: 0.0906
Used Parameters: lstm_sizes=[32, 16], hidden_size=128, dropout_rate=0.3, learning_rate=0.01, batch_size=8, pos_weight=1
Early stopping activated!
Total training time: 9.20 seconds
Epoch 4, Train Loss: 0.1261, Valid Loss: 0.1197
Used Parameters: lstm_sizes=[32, 16], hidden_size=128, dropout_rate=0.3, learning_rate=0.01, batch_size=8, pos_weight=1
Epoch 8, Train Loss: 0.1200, Valid Loss: 0.0988
Used Parameters: lstm_sizes=[32, 16], hidden_size=128, dropout_rate=0.3, learning_rate=0.01, batch_size=8, pos_weight=1
Early stopping activated!
Total training time: 4.61 seconds
Epoch 4, Train Loss: 0.5866, Valid Loss: 0.5083
Used Parameters: lstm_sizes=[32, 16], hidden_size=128, dropout_rate=0.3, learning_rate=0.01, batch_size=8, pos_weight=10
Epoch 8, Train Loss: 0.5618, Valid Lo